In [1]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img  # Updated import statement
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np


In [2]:
TRAIN_DIR = '/Users/akshay/Documents/CVIP/project final/images/train'
TEST_DIR = '/Users/akshay/Documents/CVIP/project final/images/validation'

In [3]:
import os
import pandas as pd

def createdataframe(dir):
    image_paths = []
    labels = []

    # Iterate through each label directory
    for label in os.listdir(dir):
        if not label.startswith('.'):  # Skip system files
            for imagename in os.listdir(os.path.join(dir, label)):
                if not imagename.startswith('.'):  # Skip system files
                    image_paths.append(os.path.join(dir, label, imagename))
                    labels.append(label)

    # Create a DataFrame from image paths and labels
    df = pd.DataFrame({'image': image_paths, 'label': labels})
    return df

# Replace 'TRAIN_DIR' with your actual directory path
TRAIN_DIR = '/Users/akshay/Documents/CVIP/project final/images/train'

# Create the DataFrame
train = createdataframe(TRAIN_DIR)


In [4]:
print(train)

                                                   image  label
0      /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  happy
1      /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  happy
2      /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  happy
3      /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  happy
4      /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  happy
...                                                  ...    ...
21072  /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  angry
21073  /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  angry
21074  /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  angry
21075  /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  angry
21076  /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  angry

[21077 rows x 2 columns]


In [5]:
import os
import pandas as pd

def createdataframe(dir):
    image_paths = []
    labels = []

    # Iterate through each label directory
    for label in os.listdir(dir):
        if not label.startswith('.'):  # Skip system files
            for imagename in os.listdir(os.path.join(dir, label)):
                if not imagename.startswith('.'):  # Skip system files
                    image_paths.append(os.path.join(dir, label, imagename))
                    labels.append(label)

    # Create a DataFrame from image paths and labels
    df = pd.DataFrame({'image': image_paths, 'label': labels})
    return df

# Replace 'TEST_DIR' with your actual directory path for test images
TEST_DIR = '/Users/akshay/Documents/CVIP/project final/images/validation'

# Create the DataFrame for test data
test = createdataframe(TEST_DIR)


In [6]:
print(test)
print(test['image'])

                                                  image  label
0     /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  happy
1     /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  happy
2     /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  happy
3     /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  happy
4     /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  happy
...                                                 ...    ...
5135  /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  angry
5136  /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  angry
5137  /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  angry
5138  /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  angry
5139  /Users/akshay/Documents/CVIP/Face_Emotion_Reco...  angry

[5140 rows x 2 columns]
0       /Users/akshay/Documents/CVIP/Face_Emotion_Reco...
1       /Users/akshay/Documents/CVIP/Face_Emotion_Reco...
2       /Users/akshay/Documents/CVIP/Face_Emotion_Reco...
3       /Users/akshay/Documents/CVIP/Face_Emo

In [7]:
from tqdm.notebook import tqdm

In [8]:
from keras.preprocessing.image import load_img, img_to_array

# Set the desired dimensions for your images
image_height = 48
image_width = 48

# Function to extract features
def extract_features(images):
    features = []
    for image_path in tqdm(images):
        img = load_img(image_path, color_mode='grayscale', target_size=(image_height, image_width))
        img_array = img_to_array(img)
        features.append(img_array)
    return np.array(features)

# Usage
train_features = extract_features(train['image'])


  0%|          | 0/21077 [00:00<?, ?it/s]

In [9]:
from keras.preprocessing.image import load_img, img_to_array

# Function to extract features
def extract_features(images, image_height, image_width):
    features = []
    for image_path in tqdm(images):
        img = load_img(image_path, color_mode='grayscale', target_size=(image_height, image_width))
        img_array = img_to_array(img)
        features.append(img_array)
    return np.array(features)

# Usage
test_features = extract_features(test['image'], image_height, image_width)


  0%|          | 0/5140 [00:00<?, ?it/s]

In [10]:
x_train = train_features/255.0
x_test = test_features/255.0

In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [13]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [14]:
y_train = to_categorical(y_train,num_classes = 4)
y_test = to_categorical(y_test,num_classes = 4)

In [15]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Define the input layer separately
input_shape = (48, 48, 1)
input_layer = Input(shape=input_shape)

# Convolutional layers
conv1 = Conv2D(128, kernel_size=(3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
drop1 = Dropout(0.4)(pool1)

conv2 = Conv2D(256, kernel_size=(3, 3), activation='relu')(drop1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
drop2 = Dropout(0.4)(pool2)

conv3 = Conv2D(512, kernel_size=(3, 3), activation='relu')(drop2)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
drop3 = Dropout(0.4)(pool3)

conv4 = Conv2D(512, kernel_size=(3, 3), activation='relu')(drop3)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
drop4 = Dropout(0.4)(pool4)

# Flatten layer
flatten = Flatten()(drop4)

# Fully connected layers
dense1 = Dense(512, activation='relu')(flatten)
drop5 = Dropout(0.4)(dense1)

dense2 = Dense(256, activation='relu')(drop5)
drop6 = Dropout(0.3)(dense2)

# Output layer
output = Dense(4, activation='softmax')(drop6)

# Create the model
model = Model(inputs=input_layer, outputs=output)


In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [19]:
from PIL import Image
import numpy as np

def preprocess_images(image_paths, target_size=(48, 48)):
    images = []
    for image_path in image_paths:
        img = Image.open(image_path).convert('L')  # Convert to grayscale
        img = img.resize(target_size)  # Resize to (48, 48)
        img_array = np.array(img)
        images.append(img_array)
    return np.array(images)


In [20]:
import os

def get_image_paths(directory):
    image_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):  # Adjust file extensions accordingly
                image_paths.append(os.path.join(root, file))
    return image_paths

# Get the paths for training and testing images
train_image_paths = get_image_paths('/Users/akshay/Documents/CVIP/project final/images/train')
test_image_paths = get_image_paths('/Users/akshay/Documents/CVIP/project final/images/validation')

# Preprocess training and testing images
x_train_preprocessed = preprocess_images(train_image_paths)
x_test_preprocessed = preprocess_images(test_image_paths)

# Ensure that the shapes match the model's input shape
print(x_train_preprocessed.shape)  # Expected: (num_samples_train, 48, 48)
print(x_test_preprocessed.shape)  # Expected: (num_samples_test, 48, 48)


(21077, 48, 48)
(5140, 48, 48)


In [21]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 20, validation_data = (x_test,y_test)) 

Epoch 1/20
165/165 ━━━━━━━━━━━━━━━━━━━━ 500s 3s/step - accuracy: 0.3352 - loss: 1.3690 - val_accuracy: 0.3551 - val_loss: 1.3557
Epoch 2/20
165/165 ━━━━━━━━━━━━━━━━━━━━ 497s 3s/step - accuracy: 0.3482 - loss: 1.3477 - val_accuracy: 0.3667 - val_loss: 1.3201
Epoch 3/20
165/165 ━━━━━━━━━━━━━━━━━━━━ 507s 3s/step - accuracy: 0.3792 - loss: 1.3106 - val_accuracy: 0.4846 - val_loss: 1.1813
Epoch 4/20
165/165 ━━━━━━━━━━━━━━━━━━━━ 497s 3s/step - accuracy: 0.4561 - loss: 1.2119 - val_accuracy: 0.5198 - val_loss: 1.1009
Epoch 5/20
165/165 ━━━━━━━━━━━━━━━━━━━━ 532s 3s/step - accuracy: 0.4976 - loss: 1.1306 - val_accuracy: 0.5648 - val_loss: 1.0142
Epoch 6/20
165/165 ━━━━━━━━━━━━━━━━━━━━ 514s 3s/step - accuracy: 0.5284 - loss: 1.0825 - val_accuracy: 0.5652 - val_loss: 0.9986
Epoch 7/20
165/165 ━━━━━━━━━━━━━━━━━━━━ 289s 2s/step - accuracy: 0.5551 - loss: 1.0365 - val_accuracy: 0.5934 - val_loss: 0.9583
Epoch 8/20
165/165 ━━━━━━━━━━━━━━━━━━━━ 305s 2s/step - accuracy: 0.5633 - loss: 1.0171 - val_accu

In [28]:
from keras.models import model_from_json

# Save model architecture as JSON file
model_json = model.to_json()
with open("emotiondetector.json", "w") as json_file:
    json_file.write(model_json)

# Save model weights with the filename ending in `.weights.h5`
model.save("emotiondetector.h5")



In [7]:
import os

file_path = '/Users/akshay/Documents/CVIP/project final/images/emotiondetector.h5'

if os.path.exists(file_path):
    print("File exists.")
else:
    print("File does not exist. Please verify the file path.")

from tensorflow.keras.models import load_model

# Load the model
model = load_model('/Users/akshay/Documents/CVIP/project final/emotiondetector.h5')

# Display the model summary
model.summary()

File does not exist. Please verify the file path.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 48, 48, 1)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d (Conv2D)                 │ (None, 46, 46, 128)       │      1,280 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 128)       │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout (Dropout)               │ (None, 23, 23, 128)       │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 256)       │    295,168 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 256)       │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 256)       │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 512)         │  1,180,160 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 512)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 512)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_3 (Conv2D)               │ (None, 2, 2, 512)         │  2,359,808 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 1, 1, 512)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_3 (Dropout)             │ (None, 1, 1, 512)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten (Flatten)               │ (None, 512)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 512)               │    262,656 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_4 (Dropout)             │ (None, 512)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_1 (Dense)                 │ (None, 256)               │    131,328 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_5 (Dropout)             │ (None, 256)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_2 (Dense)                 │ (None, 4)                 │      1,028 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 4,231,430 (16.14 MB)

 Trainable params: 4,231,428 (16.14 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)